<a href="https://colab.research.google.com/github/DmitryMok/od_helper/blob/main/prepare_for_od.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Модуль для анализа и конвертации датасета изображения

1. Анализ датасета (размеры и ориентация изображений)
2. Анализ аннотаций (наличие аннотаций для всех изображений)
3. Конвертация в формат yolo
4. Фильтрация изображений (по размерам, количеству изображений и т.п.)

# Анализ датасета

In [ ]:
!git clone https://github.com/DmitryMok/wine_helper # вспомогательные функции
from wine_helper.prepare_data_helper import *

Cloning into 'wine_helper'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 54 (delta 15), reused 34 (delta 9), pack-reused 0
Unpacking objects: 100% (54/54), done.


In [ ]:
import time
import random as r

In [ ]:
!pip install xmltodict  # устанавливаем библиотеку для работы с xml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import xmltodict

# Классы

In [ ]:
# класс для работы с датасетом
class DataSet:
  '''
  класс содержит пути к файлам изображений и разметки
  выводит статистику: количество файлов, разрешения файлов
  разбивает на train и val
  '''
  def __init__(self, img_dirs, lbl_dirs):
    self.img_dirs = img_dirs  # !list of train/val/test dir
    self.lbl_dirs = lbl_dirs

    # os.path.join(img_dir, lbl_dir)

    # self.img_lst = os.listdir(img_dir)
    # self.lbl_lst = os.listdir(lbl_dir)

  def get_stat(self):
    # выводит количество файлов и список расширений файлов
    print('Папки с изображениями:\n')
    commonpath = os.path.commonpath(self.img_dirs)
    print(commonpath) # общая часть папок, чтобы отобразить структуру
    # выводим информацию в каждой папке
    for img_dir in self.img_dirs:
      img_lst = [f for f in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, f))]
      print('  |')
      print('  +-- ',os.path.relpath(img_dir, start=commonpath), ' [кол-во файлов - ',
            len(img_lst), ', расширения: ', *{img_name.split('.')[-1] for img_name in img_lst if len(img_name.split('.'))>1},']', sep='')

    print('\n\nПапки с файлами разметки:\n')
    commonpath = os.path.commonpath(self.lbl_dirs)
    print(commonpath) # общая часть папок, чтобы отобразить структуру
    # выводим информацию в каждой папке
    for lbl_dir in self.lbl_dirs:
      lbl_lst = [f for f in os.listdir(lbl_dir) if os.path.isfile(os.path.join(lbl_dir, f))]
      print('  |')
      print('  +-- ',os.path.relpath(lbl_dir, start=commonpath), ' [кол-во файлов - ',
            len(lbl_lst), ', расширения: ', *{lbl_name.split('.')[-1] for lbl_name in lbl_lst if len(lbl_name.split('.'))>1},']', sep='')

  # расширенная статистика по классам и объектам (количество, размеры, примеры классов)
  def get_class_stat(self,img_list, bbox_list_tmp, img_dir, wh_offset = 1):  #
    # выводит статистику по классам
    # автоматически переводит относительные координаты в абсолютные
    # на вход массивы: [image_name], [class, x, y, w, h] и путь к изображениям
    # bbox_list - абсолютные координаты
    # wh_offset - 0 - координаты считаем от центра, 1 - координаты угла
    # self.bbox_list - относительные координаты (ToDo в отдельном методе)

    bbox_list = bbox_list_tmp.copy()  # чтобы не затереть при переводе в абосолютные координаты
    x,y,w,h = bbox_list[0,1:]
    # если координаты относительные, переводим в абсолютные
    self.bbox_list = bbox_list.copy()
    # читаем изображения для получения размеров
    flag = True
    new_time = time.time()
    img_lst = np.unique(img_list, axis=0) # # уникальный список изображений, чтобы не считывать неколько раз одно изображение
    if (x < 1) and (y < 1):
      print(f'Требуется перевод относительных координат в абсолютные ({len(np.unique(img_list, axis=0))} файлов)')
      for i, im in enumerate(img_lst):
        # индексы относящиеся к этому изображению
        idx = np.where(img_list == im)
        img = cv2.imread(os.path.join(img_dir,im))
        for id in idx[0]:
          bbox_list[id,[1,3]] *= img.shape[1]
          bbox_list[id,[2,4]] *= img.shape[0]
      # for i in range(len(np.unique(img_list, axis=0))):
      #   img = cv2.imread(os.path.join(img_dir,img_list[i]))
      #   bbox_list[i,[1,3]] *= img.shape[1]
      #   bbox_list[i,[2,4]] *= img.shape[0]

        if flag and time.time()-new_time > 1:
          print('\n...идет обработка, подождите еще примерно', round((time.time()-new_time)*len(img_lst)/(i+1)), 'сек\n')
          flag = False
      # else:   # не проверялась
    #   self.bbox_list[:,[1,3]] /= img.shape[1]
    #   self.bbox_list[:,[2,4]] /= img.shape[0]

    # 1. список всех классов
    self.class_list = np.unique(bbox_list[:,0])
    print('Классы:', *self.class_list)

    # 2. баланс классов
    self.class_counts = [len(bbox_list[bbox_list[:,0]==cl]) for cl in self.class_list]
    # print(class_counts)
    D = self.class_counts
    with plt.style.context('dark_background'):
      plt.bar(range(len(D)), D, align='center')
      plt.xticks(range(len(D)), self.class_list)
      plt.title('Распределение количества классов')
      plt.show()
      print()

    # 3. распределение количества объектов на изображениях
    _, count_obj = np.unique(img_list[:], return_counts=True) # считаем количество объектов на каждом изображении
    # повторы уникальных значений
    _, count_obj = np.unique(count_obj, return_counts=True)   # считаем уникальные повторы
    # print(np.unique(img_list[:]))
    D = count_obj
    with plt.style.context('dark_background'):
      plt.bar(range(len(D)), D, align='center')
      plt.xticks(range(len(D)), range(1,len(D)+1))
      plt.title('Распределение кол. объектов на изображениях')
      plt.show()

    # 4. выводим рандомно примеры каждого из классов
    for cl in self.class_list:
      cl_idx_num = 5 # количесво примеров класса для вывода
      print('#'*50)
      print('КЛАСС -', cl)
      print('#'*50)

      # показываем на графике статистику размеров bbox

      # индексы нужного класса
      idx_classes = np.where(bbox_list[:,0]==cl)[0]

      # индексы нужного размера (5 самых крупных или крупнее 100, но не крупнее 400)
      idx_bb_size_min = np.where(bbox_list[:,4] >= min(100, np.sort(bbox_list[idx_classes][:,4])[-min(cl_idx_num,len(bbox_list[idx_classes]))]))[0]
      idx_bb_size_max = np.where(bbox_list[:,4] <= 400)[0]  # для ограничения максиального размера

      # индексы из пересечения индексов нужного класса и размеров (попадают объекты с высотой от 100 до 400 или самые крупные, если меньше 100)
      idx = list(set(idx_classes) & set(idx_bb_size_min) & set(idx_bb_size_max))
      # print(bbox_list[idx])

      with plt.style.context('dark_background'):
        plt.title(f'Размеры bounding box, class={cl}, {len(idx_classes)}шт')
        # plt.plot(wh_lst[1::2], wh_lst[0::2],':o')
        # plt.show()

        w, h = bbox_list[bbox_list[:,0]==cl][:,3], bbox_list[bbox_list[:,0]==cl][:,4]

        wh_zip = list(zip(w,h)) # создаем пары, чтобы работать с ними в словаре
        d = {d:wh_zip.count(d) for d in set(wh_zip)}
        # print(d.values())
        s = [s for s in d.values()]
        # s = [20*4**n for n in range(len(x))]
        plt.scatter([x for x,y in d.keys()],[y for x,y in d.keys()],s=s)
        plt.show()
      # 5 случайных интексов для вывода
      samp_idx = np.random.choice(idx, np.min([cl_idx_num, len(idx)]), replace=False)
      # print('samp_idx', samp_idx)
      for i in samp_idx:
        # print(samp_idx, img_list.shape)
        img = cv2.imread(os.path.join(img_dir,img_list[i]))
        print(os.path.join(img_dir,img_list[i]))
        x,y,w,h = bbox_list[i][1:]

        if (w > 1) and (h > 1):
          print(bbox_list[i])
          # print(x,y,w,h, img.shape)
          if wh_offset:
            img_class = img[int(y-h/2):int(y+h/2), int(x-w/2):int(x+w/2)]
          else:
            img_class = img[int(y):int(y+h), int(x):int(x+w)]
          cv2_imshow(img_class)

  def get_img_idx(self, img_list, idx):
    # Функция возвращает индексы списка изображений, где есть нужные классы
    # 1. находим индексы, где есть нужный класс
    img_filtered = np.unique(img_list[idx])
    idx = np.where(np.in1d(img_list, img_filtered))[0]
    return idx, img_filtered  # возвращаем индексы с нужными изображениями и список нужных изображений

  def get_filt_classes(self, img_list, bbox_list, img_dir, inc_cl=None, exc_cl=None, mis_sq=0, max_sq=10e7):
    '''
    функция возвращает индексы для определенных классов и параметров
    :img_list: список названий файлов
    :bbox_list: список файлов с аннотацией [class x y w h]. Координаты могут быть относительные или абсолютные
    :inc_cl: классы, которые надо включить
    :exc_cl: классы, которые надо исключить
    :mis_sq: минимальная площать bbox
    :max_sq: максиальная площать bbox
    :return: список индексов подходящик под фильтр
    '''

    # 1. список всех классов
    self.class_list = np.unique(bbox_list[:,0])

    print('Список классов:', self.class_list)
    idx_set_list = [] # здесь будут списки индексов, пересечение которых определим
    # idx = 0

    if inc_cl:
      if not sum([cl in self.class_list for cl in inc_cl]):
        print(f'ERROR: класс {inc_cl} не входит в список классов {self.class_list}')
      # 1. находим индексы, где встречается нужный класс
      idx = np.where(np.in1d(bbox_list[:,0], inc_cl))[0]
      # 2. фильтруем изображения, чтобы остались только с нужными индексами
      idx, img_filtered = self.get_img_idx(img_list, idx)

    # исключаем изображения с ненужными классами
    if exc_cl:
      if not sum([cl in self.class_list for cl in exc_cl]):
        print(f'ERROR: класс {exc_cl} не входит в список классов {self.class_list}')
      # 1. находим индексы, где встречается нужный класс
      idx = np.where(np.in1d(bbox_list[:,0], exc_cl))[0]
      # 2. фильтруем изображения, чтобы остались только с нужными индексами
      idx, img_filtered = self.get_img_idx(img_list, idx)
      # 3. вычитаем найденные индексы и названия файлов, чтобы исключить
      # print('idx', idx)
      idx = np.setdiff1d(np.arange(len(img_list)), idx)
      img_filtered = np.unique(img_list[idx])

    # исключаем изображения с мелкими объектами
    if mis_sq:
      # 1. считаем площадь объектов
      sq_list = bbox_list[:,3] * bbox_list[:,4]
      # 2. находим индексы, где встречается нужный класс
      idx = np.where(sq_list < mis_sq)[0]
      # 3. фильтруем изображения, чтобы остались только с нужными индексами
      idx, img_filtered = self.get_img_idx(img_list, idx)
      # 4. вычитаем найденные индексы и названия файлов, чтобы исключить изображения с мелкими объектами
      # print('idx', idx)
      idx = np.setdiff1d(np.arange(len(img_list)), idx)
      img_filtered = np.unique(img_list[idx])

      # print(*sq_list[idx])


    self.idx = idx
    print(f'из {len(np.unique(img_list))} изображений соответствует фильтру {len(img_filtered)} изображений ({len(bbox_list)}/{len(self.idx)})')
    print(f'включены классы {inc_cl}, исключены {exc_cl}, мин. площадь {mis_sq}, макс. площадь {max_sq}')
    print(f'\nостались классы {np.unique(bbox_list[idx,0])}')

    return self.idx



---

Блок тестов

In [ ]:
# !cp /content/drive/MyDrive/Project_people/dataset_land/vis_drone19_sample.zip /content/

In [ ]:
# !unzip -q vis_drone19_sample.zip -d /content

конец блока тестов

---



# Функциии

**вывод несколько изображений из папки**

In [ ]:
def draw_imgs_from_path(dir, num=8):
  '''
  Draws images from the folder as a grid (4 cols x num / 4 rows)
  :dir: path to folder with images
  :num: number of images (default = 16)
  '''
  # получим список с именами всех картинок, находящихся в папке pic
  pictures = os.listdir(dir)
  # Упорядочим список
  pictures = sorted(pictures)
  # Создадим фигуру размером 16 на 4 дюйма
  pic_box = plt.figure(figsize=(32,5*(num//4+1)))

  # Поочередно считываем в переменную picture имя изображения из списка pictures. В переменную i записываем номер итерации
  for i, picture in enumerate(pictures[:num]):
      # считываем изображение в picture
      picture = cv2.imread(os.path.join(dir,picture))
      # конвертируем BGR изображение в RGB
      picture = cv2.cvtColor(picture, cv2.COLOR_BGR2RGB)
      # добавляем ячейку в pix_box для вывода текущего изображения
      pic_box.add_subplot(num//4+1,4,i+1)
      plt.title(pictures[i] + '\n' + str(picture.shape))
      plt.imshow(picture)
      # отключаем отображение осей
      plt.axis('off')
  # выводим все созданные фигуры на экран
  plt.show()


In [ ]:
def check_img_dir(dir):
  '''
  Counts images in the folder
  shows file extensions and a graph of image sizes
  :dir: path to folder with images
  '''
  wh_lst = []   # массив для размеров изображений
  img_lst = os.listdir(dir)
  flag = True # флаг мониторинга контроля ожидания
  print('Количество файлов в папке:', len(img_lst))
  print('Расширения:', *{img_name.split('.')[-1] for img_name in img_lst})
  new_time = time.time()
  for i,img_name in enumerate(img_lst[:]):
    tmp_img = cv2.imread(os.path.join(dir,img_name))
    wh_lst.extend(tmp_img.shape[:2])
    # print(img_name, tmp_img.shape[:2])
    if flag and time.time()-new_time > 1:
      print('\n...идет обработка, подождите еще примерно', round((time.time()-new_time)*len(img_lst)/(i+1)), 'сек\n')
      flag = False

  with plt.style.context('dark_background'):
    plt.title('Размеры изображений')
    # plt.plot(wh_lst[1::2], wh_lst[0::2],':o')
    # plt.show()

    x, y = wh_lst[1::2], wh_lst[0::2]

    wh_zip = list(zip(x,y)) # создаем пары, чтобы работать с ними в словаре
    d = {d:wh_zip.count(d) for d in set(wh_zip)}
    # print(d.values())
    s = [s for s in d.values()]
    # s = [20*4**n for n in range(len(x))]
    plt.scatter([x for x,y in d.keys()],[y for x,y in d.keys()],s=s)
    plt.show()


# check_img_dir(ds.img_dirs[0])

### Для YOLOv5

In [ ]:
# создание папок для yolo

def create_dirs_for_yolo(root_dir):
  '''
  Creates root_dir/train/images, root_dir/train/labels and the same for /val
  '''
  os.makedirs(os.path.join(root_dir,'train/images/'), exist_ok=True)
  os.makedirs(os.path.join(root_dir,'train/labels/'), exist_ok=True)

  os.makedirs(os.path.join(root_dir,'val/images/'), exist_ok=True)
  os.makedirs(os.path.join(root_dir+'val/labels/'), exist_ok=True)

In [ ]:
# создаем custom.yaml
def create_yaml(fname='custom.yaml', classes={0:'person'}, main_dir='/content/dataset/', dirs=['train','val']):
  '''
  Creates custom.yaml for yolov5
  :fname: yaml file name
  :classes: dictionary of all classes (default {0:'person'})
  :main_dir: path to root folder
  :dirs: train and val subfolders
  '''
  file = open(fname, 'w')  # файл будет создан, если отсутствует

  file.write(f"train: {main_dir+dirs[0]}/\n")
  file.write(f"val: {main_dir+dirs[1]}/\n\n")

  file.write(f"nc: {len(classes)}\n\n")

  file.write(f"names: {list(classes.values())}")
  # names: ['person']
  file.close()

In [ ]:
# 17.10.22

def mapping_classes(class_mapping, dir_label, resave=0):
  '''
  Функция меняет класс в аннотации yolo. Например меняет все классы 4 на класс 0.
  На вход передается словать {старый класс:новый класс} и путь к файлам аннотации
  resave=1 - перезапись сущ. файлов. 0 - только вывод на печать
  '''

  bbox_list = []

  for lb_fname in os.listdir(dir_label)[:]:
    lb = np.loadtxt(os.path.join(dir_label, lb_fname))
    # проверяем, если размерность = 1 добавляем размерность, иначе ошибка при конвертации в массив numpy
    if lb.shape[0]: # если файл разметки непустой
      if len(np.array(lb).shape) != 2:
        lb = [lb]
        # print(im,len(np.array(lb).shape))
      # bbox_list.extend(lb)
      lb = np.array(lb)

      # замена классов на целевые
      class_new = [class_mapping[cl] for cl in lb[:][:,0]]
      annot_data = np.column_stack([class_new, lb[:][:,1:]])

      if resave:
        np.savetxt(os.path.join(dir_label,lb_fname), annot_data, fmt='%s', delimiter=' ')
      else:
        print('было', lb, '\nстало', annot_data)

In [ ]:
print('\nфункции успешно загружены')
print('используйте функции:')
print('draw_imgs_from_path - для вывода сетки изображений из папки (файлы сортируются по имени)')
print('check_img_dir - чтобы получить количество изображений, типы расширений и разброс размеров на графике')
print('create_dirs_for_yolo - подготовить папки для датасета на котором будет обучаться yolo')
print('create_yaml - для создания файла yaml yolo с описанием путей и классов')




функции успешно загружены
используйте функции:
draw_imgs_from_path - для вывода сетки изображений из папки (файлы сортируются по имени)
check_img_dir - чтобы получить количество изображений, типы расширений и разброс размеров на графике
create_dirs_for_yolo - подготовить папки для датасета на котором будет обучаться yolo
create_yaml - для создания файла yaml yolo с описанием путей и классов
